In [1]:
#Prep Data for Model

In [2]:
import pandas as pd
import numpy as np

## 1. Get Demand Data

In [3]:
revenue = pd.read_excel('Revenue.xlsx')

In [4]:
#Split Asia Pacific and Latin America into Separate Regions
apac_latam = revenue[revenue['Region'] == 'Asia Pacific & Latin America']
apac_gdp_perc = 0.733 #looked up online
latam_gdp_perc = 1 - apac_gdp_perc
apac_latam['Asia Pacific'] = apac_latam['Revenue ($M)'] * apac_gdp_perc
apac_latam['Latin America'] = apac_latam['Revenue ($M)'] * latam_gdp_perc
apac_latam = apac_latam.drop(columns = ['Region','Revenue ($M)'])
apac_latam = pd.melt(apac_latam, id_vars=['Year', 'Category'], var_name='Region', value_name='Revenue ($M)')

/var/folders/56/bfw_b7050wxcy69jmv2db0040000gn/T/ipykernel_11851/2130447476.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apac_latam['Asia Pacific'] = apac_latam['Revenue ($M)'] * apac_gdp_perc
/var/folders/56/bfw_b7050wxcy69jmv2db0040000gn/T/ipykernel_11851/2130447476.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apac_latam['Latin America'] = apac_latam['Revenue ($M)'] * latam_gdp_perc


In [5]:
#Merge Asia Pacific and Latin America back in
revenue = revenue[revenue['Region'] != 'Asia Pacific & Latin America']
revenue = pd.concat([revenue, apac_latam], ignore_index=True, axis=0)

In [6]:
rev_by_region = revenue.groupby(['Year', 'Region']).sum('Revenue ($M)')
rev_by_region = rev_by_region.reset_index()
rev_by_region = rev_by_region.rename(columns = {'Revenue ($M)': 'Yr_Region_Total'})

In [7]:
rev_by_year = revenue.groupby('Year').sum('Revenue ($M)')
rev_by_year = rev_by_year.reset_index()
rev_by_year = rev_by_year.rename(columns = {'Revenue ($M)': 'Yr_Total'})

In [8]:
revenue = revenue.merge(rev_by_region,
                        how = 'left',
                        on = ['Year', 'Region'])
revenue = revenue.merge(rev_by_year,
                        how = 'left',
                        on = 'Year')

In [9]:
revenue = revenue[revenue['Category'] == 'Footwear']
revenue['perc_rev_footware'] = revenue['Revenue ($M)']/revenue['Yr_Region_Total']
revenue['perc_rev_in_region'] = revenue['Yr_Region_Total']/revenue['Yr_Total']

In [10]:
avg_cost_shoes = 116.5 #based on nike website

In [11]:
revenue['shoes_made'] = revenue['Revenue ($M)'] * 1e6 / avg_cost_shoes
revenue['shoes_made'] = revenue['shoes_made'].round(decimals=0)

In [12]:
demand = revenue[['Year', 'Region', 'shoes_made']]

In [13]:
demand = demand.pivot(index='Region', columns='Year', values='shoes_made')

In [14]:
demand.to_csv('demand.csv')

In [15]:
#will need for workers
shoes_per_year  = demand.sum(axis = 0).to_list()
shoes_per_year.reverse()

In [16]:
rev = revenue[['Year', 'Region', 'Revenue ($M)']]
rev = rev.pivot(index='Region', columns='Year', values='Revenue ($M)')
rev.to_csv('rev.csv')

## 2. Get Availability

### a. Read in Workers Data (2023)

In [17]:
workers = pd.read_excel('imap_export.xls', skiprows = 1)

In [18]:
workers = workers[workers['Product Type Type'] == 'Footwear']
wokers = workers[workers['Factory Type'] == 'FINISHED GOODS']


In [19]:
workers = workers[['Region', 'Total Workers']].groupby('Region').sum()

In [20]:
workers.head()

,Total Workers
Region,
AMERICAS,16925
EMEA,2514
N ASIA,99736
S ASIA,264761
SE ASIA,348364


### b. Prep Rev Data to Estimate Workers Each Year

In [21]:
rev_by_year['priorYr'] = rev_by_year['Year']-1

In [22]:
rev_by_year = rev_by_year.merge(rev_by_year,
                                how = 'left',
                                left_on = 'priorYr',
                                right_on = 'Year')

In [23]:
rev_by_year = rev_by_year.drop(columns = ['priorYr_x', 'priorYr_y', 'Year_y'])
rev_by_year = rev_by_year.drop(0)
rev_by_year = rev_by_year.rename(columns = {'Year_x' : 'Year',
                                           'Yr_Total_x': 'Yr_Total',
                                           'Yr_Total_y' : 'Prior_Yr_Total'})

In [24]:
rev_by_year['growth'] = (rev_by_year['Yr_Total']- rev_by_year['Prior_Yr_Total'])/rev_by_year['Prior_Yr_Total']

In [25]:
rev_by_year = rev_by_year.sort_values('Year', ascending = False)

### c. Get Workers Each Year

In [26]:
workers_per_year = workers

In [27]:
for i in range(0, 7):
    year = rev_by_year['Year'].iloc[i] - 1
    col_year = 'yr_' + str(year)
    last_column_name = workers_per_year.columns[-1]
    growth = rev_by_year['growth'].iloc[i]
    workers_per_year[col_year] = workers_per_year[last_column_name] * (1.0 - growth) ** 0.5

In [28]:
workers_per_year = workers_per_year.rename(columns = {'Total Workers' : 'yr_2023'})

In [29]:
workers_per_year

,yr_2023,yr_2022,yr_2021,yr_2020,yr_2019,yr_2018,yr_2017,yr_2016
Region,,,,,,,,
AMERICAS,16925,16069.015157,15671.379968,14109.684516,14417.175141,13822.515598,13333.115306,12949.517462
EMEA,2514,2386.854009,2327.790206,2095.819608,2141.493548,2053.164208,1980.469830,1923.491102
N ASIA,99736,94691.834310,92348.641211,83145.849035,84957.836329,81453.613926,78569.665475,76309.191938
S ASIA,264761,251370.665995,245150.382969,220720.483439,225530.617875,216228.245334,208572.463311,202571.769138
SE ASIA,348364,330745.429609,322560.981461,290416.906164,296745.926195,284506.163890,274432.932377,266537.412171


In [30]:
seed_value = 123
np.random.seed(seed_value)

In [31]:
shoe_cols = []
for i in range(0, workers_per_year.shape[1]):
    col = workers_per_year.columns[i]
    workers_per_year[col] = workers_per_year[col].astype(int)
    total_workers = workers_per_year[col].sum()
    workers_per_year[col + '_perc'] = workers_per_year[col] / total_workers
    workers_per_year[col + '_shoes'] = workers_per_year[col + '_perc'] * shoes_per_year[i-1]
    workers_per_year[col + '_shoes'] = (workers_per_year[col + '_shoes'] * (np.random.rand()/4)).astype(int)
    shoe_cols.append(col + '_shoes')

In [32]:
workers_per_year[shoe_cols]

,yr_2023_shoes,yr_2022_shoes,yr_2021_shoes,yr_2020_shoes,yr_2019_shoes,yr_2018_shoes,yr_2017_shoes,yr_2016_shoes
Region,,,,,,,,
AMERICAS,686396,470240,327884,766158,831594,508274,1083172,715998
EMEA,101955,69823,48687,113764,123496,75494,160855,106329
N ASIA,4044814,2771018,1932198,4515007,4900450,2995260,6382945,4219408
S ASIA,10737438,7356041,5129278,11985717,13008917,7951324,16944388,11200904
SE ASIA,14127975,9678856,6748928,15770406,17116708,10462102,22294855,14737822


In [33]:
availability = workers_per_year[shoe_cols]
availability.to_csv('availability.csv')

In [34]:
availability

,yr_2023_shoes,yr_2022_shoes,yr_2021_shoes,yr_2020_shoes,yr_2019_shoes,yr_2018_shoes,yr_2017_shoes,yr_2016_shoes
Region,,,,,,,,
AMERICAS,686396,470240,327884,766158,831594,508274,1083172,715998
EMEA,101955,69823,48687,113764,123496,75494,160855,106329
N ASIA,4044814,2771018,1932198,4515007,4900450,2995260,6382945,4219408
S ASIA,10737438,7356041,5129278,11985717,13008917,7951324,16944388,11200904
SE ASIA,14127975,9678856,6748928,15770406,17116708,10462102,22294855,14737822


## 3. Get Cost Data

### a. Cost of Sales

In [35]:
cost_of_sales = pd.read_excel('CostOfSales.xlsx')

In [36]:
cost_of_sales = cost_of_sales[['Year', 'Shoes - Cost of Sales ($M)']]
cost_of_sales = cost_of_sales.rename(columns = {'Shoes - Cost of Sales ($M)' : 'Cost_of_Sales_shoes_m'})

### b. Inventory Costs

In [37]:
inventory_cost = pd.read_excel('inventory_cost.xlsx')

In [38]:
#Split Asia Pacific and Latin America into Separate Regions
apac_latam = inventory_cost[inventory_cost['Region'] == 'Asia Pacific & Latin America']
apac_gdp_perc = 0.733 #looked up online
latam_gdp_perc = 1 - apac_gdp_perc
apac_latam['Asia Pacific'] = apac_latam['Inventory Costs'] * apac_gdp_perc
apac_latam['Latin America'] = apac_latam['Inventory Costs'] * latam_gdp_perc
apac_latam = apac_latam.drop(columns = ['Region','Inventory Costs'])
apac_latam = pd.melt(apac_latam, id_vars='Year', var_name='Region', value_name='Inventory Costs')
#re-join it
inventory_cost = inventory_cost[inventory_cost['Region'] != 'Asia Pacific & Latin America']
inventory_cost = pd.concat([inventory_cost, apac_latam], ignore_index=True, axis=0)

/var/folders/56/bfw_b7050wxcy69jmv2db0040000gn/T/ipykernel_11851/3194163411.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apac_latam['Asia Pacific'] = apac_latam['Inventory Costs'] * apac_gdp_perc
/var/folders/56/bfw_b7050wxcy69jmv2db0040000gn/T/ipykernel_11851/3194163411.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  apac_latam['Latin America'] = apac_latam['Inventory Costs'] * latam_gdp_perc


### c. Get Costs

In [39]:
costs = inventory_cost.merge(cost_of_sales,
                             how = 'inner',
                             on = 'Year')

In [40]:
perc = revenue[['Year', 'Region', 'perc_rev_footware', 'perc_rev_in_region']]

In [41]:
costs = costs.merge(perc,
                    how = 'inner',
                    on = ['Year', 'Region'])

In [42]:
costs['cost_of_sales_shoes_region'] = costs['Cost_of_Sales_shoes_m'] * costs['perc_rev_footware'] * costs['perc_rev_in_region']
costs['inventory_costs_shoes'] = costs['Inventory Costs'] * costs['perc_rev_footware']

In [43]:
costs = costs.drop(columns = ['Inventory Costs', 
                              'Cost_of_Sales_shoes_m', 
                              'perc_rev_footware', 
                              'perc_rev_in_region'])

In [44]:
costs = costs.rename(columns = {'inventory_costs_shoes' : 'inventory_costs',
                               'cost_of_sales_shoes_region': 'cost_of_sales'})

In [45]:
regions = list(costs['Region'].unique())

In [46]:
region_holding_perc = []
for i in range(0, len(regions)):
    region_holding_perc.append(np.random.rand()/10 + 0.2)

In [47]:
holding_cost_perc = pd.DataFrame({'Region': regions, 'Hold_Cost_perc': region_holding_perc})

In [48]:
costs = costs.merge(holding_cost_perc,
                    how = 'inner',
                    on = 'Region')

In [49]:
costs['inventory_costs_holding'] = costs['inventory_costs'] * costs['Hold_Cost_perc']
costs['inventory_costs_goods'] = costs['inventory_costs'] * (1 - costs['Hold_Cost_perc'])

In [50]:
costs['shoes_in_inventory'] = costs['inventory_costs_goods'] * 1e6 / avg_cost_shoes

In [51]:
costs['holding_cost_per_shoe'] = costs['inventory_costs_holding'] * 1e6 / costs['shoes_in_inventory']
costs['holding_cost_per_shoe'] = costs['holding_cost_per_shoe'].round(decimals = 2)

In [52]:
costs

,Year,Region,cost_of_sales,inventory_costs,Hold_Cost_perc,inventory_costs_holding,inventory_costs_goods,shoes_in_inventory,holding_cost_per_shoe
0,2023,North America,6018.829833,2623.934746,0.248093,650.980342,1972.954404,1.693523e+07,38.44
1,2022,North America,4574.570406,2730.362557,0.248093,677.384357,2052.978200,1.762213e+07,38.44
2,2021,North America,4670.840666,1932.420048,0.248093,479.420254,1452.999793,1.247210e+07,38.44
3,2020,North America,3642.964544,1981.865023,0.248093,491.687216,1490.177807,1.279123e+07,38.44
4,2019,North America,3810.237802,1470.554647,0.248093,364.834594,1105.720053,9.491159e+06,38.44
5,2018,North America,3586.338815,1424.499495,0.248093,353.408624,1071.090871,9.193913e+06,38.44
6,2017,North America,3757.813578,1411.613565,0.248093,350.211713,1061.401852,9.110746e+06,38.44
7,2016,North America,3472.257512,1396.991466,0.248093,346.584069,1050.407396,9.016373e+06,38.44
8,2023,"Europe, Middle East, and Africa",3337.284985,1333.985691,0.239212,319.105054,1014.880637,8.711422e+06,36.63
9,2022,"Europe, Middle East, and Africa",2763.896480,1117.169324,0.239212,267.240031,849.929293,7.295530e+06,36.63


In [59]:
inventory_holding_cost = costs[['Year', 'inventory_costs_holding']].groupby('Year').sum().reset_index()

In [60]:
inventory_holding_cost.to_csv('inventory_holding_cost.csv')

In [55]:
cost_output = costs[['Year', 'Region', 'cost_of_sales']]
cost_output = cost_output.pivot(index='Region', columns='Year', values='cost_of_sales')
cost_output.to_csv('cost.csv')